# 📰 RoBERTa-Based News Fake Detection with Synonym Augmentation and Reframing

## Project Overview

This notebook implements a deep learning model for fake news detection using **RoBERTa**, enhanced with **synonym-based text augmentation** and **fine-grained reframing supervision**. The goal is to improve model robustness against adversarial paraphrasing by training on semantically equivalent but lexically altered versions of news texts.

### Key Features:
- ✅ Synonym swapping using WordNet (nouns & adjectives only)
- ✅ Triple-augmentation training: original + 2 augmented versions
- ✅ Dual-task learning: classification + fine-grained framing (binary)
- ✅ Consistency regularization via KL-divergence
- ✅ Evaluation on 4 adversarial styles: Objective, Neutral, Emotionally Triggering, Sensational
- ✅ Model checkpointing and logging

### Dependencies:
- PyTorch, Transformers, NLTK, scikit-learn
- Custom data loaders: `load_articles()` and `load_reframing()` from `utils.load_data`

Note: Ensure `utils/load_data.py` exists in the parent directory with required functions.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
from transformers.optimization import get_linear_schedule_with_warmup
from torch.optim import AdamW
import argparse
import numpy as np
import sys
import os
import nltk
import random
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import warnings

# Suppress warnings to reduce console clutter
warnings.filterwarnings("ignore")

# Add parent directory to Python path for custom utilities
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

print("🔍 Current working directory:", os.getcwd())

## 🛠️ 1. Download and Initialize NLTK Resources

We download required natural language processing datasets for synonym replacement using WordNet and POS tagging.

In [ ]:
# Download required NLTK corpora for synonym swapping and POS tagging
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

## ⚙️ 2. Command-Line Argument Parser

Configures training parameters via CLI for reproducibility and experimentation.

In [ ]:
parser = argparse.ArgumentParser(description='RoBERTa with Synonym Augmentation for Fake News Detection')
parser.add_argument('--dataset_name', default='politifact', type=str, help='Name of dataset (e.g., politifact, gossipcop)')
parser.add_argument('--model_name', default='Pretrained-LM', type=str, help='Model identifier for logging')
parser.add_argument('--iters', default=3, type=int, help='Number of training iterations (seeds)')
parser.add_argument('--batch_size', default=4, type=int, help='Batch size for training')
parser.add_argument('--n_epochs', default=5, type=int, help='Number of training epochs per iteration')

args = parser.parse_args()

# Set device to GPU if available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

# Set random seeds for reproducibility across runs
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)
print("✅ Random seeds set for reproducibility.")

## 🔁 3. Synonym Augmentation Function

Randomly replaces 1–2 nouns or adjectives in a text with synonyms from WordNet.
Only replaces words with valid synonyms and avoids replacing the same word multiple times.

In [ ]:
def synonym_swap(text, n=1):
    """
    Randomly replace n nouns or adjectives with synonyms using WordNet.
    
    Args:
        text (str): Input news text
        n (int): Number of words to replace (default: 1)
    
    Returns:
        str: Augmented text with synonyms
    """
    words = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    
    # Identify replaceable POS tags: Nouns (NN, NNS, NNP, NNPS) and Adjectives (JJ, JJR, JJS)
    replaceable_indices = [i for i, (_, tag) in enumerate(pos_tags) 
                          if tag.startswith(('NN', 'JJ'))]
    
    # If no replaceable words, return original text
    if not replaceable_indices:
        return text
    
    # Limit replacements to available words
    n = min(n, len(replaceable_indices))
    selected_indices = random.sample(replaceable_indices, n)
    
    for idx in selected_indices:
        word = words[idx]
        synonyms = []
        
        # Collect unique synonyms from WordNet
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonym = lemma.name().replace('_', ' ')
                if synonym != word and synonym not in synonyms:
                    synonyms.append(synonym)
        
        # Replace word with a random synonym if available
        if synonyms:
            words[idx] = random.choice(synonyms)
    
    return ' '.join(words)

## 📦 4. Training Dataset Class: NewsDatasetAug

Custom PyTorch Dataset that loads:
- Original text
- Two synonym-augmented versions (random 1–2 swaps)
- Labels and fine-grained framing labels (for consistency loss)

Uses RoBERTa tokenizer to encode all versions with padding and truncation.

In [ ]:
class NewsDatasetAug(Dataset):
    """
    Dataset for training with synonym augmentation and fine-grained framing supervision.
    Each sample includes original text + 2 augmented versions with corresponding labels.
    """
    def __init__(self, texts, aug_texts1, aug_texts2, labels, fg_label, aug_fg1, aug_fg2, tokenizer, max_len):
        self.texts = texts
        # Apply synonym swapping on-the-fly for stochastic augmentation
        self.aug_texts1 = [synonym_swap(text, n=random.randint(1, 2)) for text in aug_texts1]
        self.aug_texts2 = [synonym_swap(text, n=random.randint(1, 2)) for text in aug_texts2]
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.labels = np.array(labels)
        self.fg_label = np.array(fg_label)
        self.aug_fg1 = np.array(aug_fg1)
        self.aug_fg2 = np.array(aug_fg2)

    def __getitem__(self, item):
        text = self.texts[item]
        aug_text1 = self.aug_texts1[item]
        aug_text2 = self.aug_texts2[item]
        label = self.labels[item]
        fg_label = self.fg_label[item]
        aug_fg1 = self.aug_fg1[item]
        aug_fg2 = self.aug_fg2[item]
        
        # Encode original text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt'
        )

        # Encode first augmented version
        aug1_encoding = self.tokenizer.encode_plus(
            aug_text1,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt'
        )

        # Encode second augmented version
        aug2_encoding = self.tokenizer.encode_plus(
            aug_text2,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'input_ids_aug1': aug1_encoding['input_ids'].flatten(),
            'input_ids_aug2': aug2_encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'attention_mask_aug1': aug1_encoding['attention_mask'].flatten(),
            'attention_mask_aug2': aug2_encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long),
            'fg_label': torch.tensor(fg_label, dtype=torch.float),
            'fg_label_aug1': torch.tensor(aug_fg1, dtype=torch.float),
            'fg_label_aug2': torch.tensor(aug_fg2, dtype=torch.float),
        }

    def __len__(self):
        return len(self.texts)

## 📦 5. Evaluation Dataset Class: NewsDataset

Simplified dataset class for evaluating on original (non-augmented) test sets.
Used for reporting performance on clean, adversarial, and reframed test splits.

In [ ]:
class NewsDataset(Dataset):
    """
    Dataset for evaluation on original texts only.
    Used for testing on clean, adversarial, and reframed test sets.
    """
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = np.array(labels)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'news_text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

    def __len__(self):
        return len(self.texts)

## 🧠 6. RoBERTa Classifier Model Architecture

Dual-output RoBERTa classifier:
- Primary output: 4-class fake news classification
- Secondary output: Binary fine-grained framing prediction (e.g., objective vs. sensational)

Uses dropout and linear projections for regularization and task-specific heads.

In [ ]:
class RobertaClassifier(nn.Module):
    """
    RoBERTa-based classifier with dual outputs:
    - Multi-class classification (4 classes: true, false, etc.)
    - Binary framing classification (e.g., objective/neutral/emotional/sensational)
    """
    def __init__(self, n_classes):
        super(RobertaClassifier, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(p=0.5)
        self.fc_out = nn.Linear(self.roberta.config.hidden_size, n_classes)  # 4-class output
        self.binary_transform = nn.Linear(self.roberta.config.hidden_size, 2)  # Binary framing output

    def forward(self, input_ids, attention_mask):
        # Get RoBERTa pooled output
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_outputs = outputs[1]  # [CLS] token embedding
        
        # Apply dropout for regularization
        pooled_outputs = self.dropout(pooled_outputs)
        
        # Dual outputs
        class_logits = self.fc_out(pooled_outputs)     # 4-class classification
        framing_logits = self.binary_transform(pooled_outputs)  # Binary framing
        
        return class_logits, framing_logits

## 🚚 7. Data Loaders: Training and Evaluation

Factory functions to create PyTorch DataLoaders for:
- Training: with synonym-augmented triplets
- Evaluation: on original test sets (clean + 4 adversarial variants)

In [ ]:
def create_train_loader(contents, contents_aug1, contents_aug2, labels, fg_label, aug_fg1, aug_fg2, tokenizer, max_len, batch_size):
    """
    Creates a DataLoader for training with original + 2 synonym-augmented samples.
    """
    ds = NewsDatasetAug(
        texts=contents,
        aug_texts1=contents_aug1,
        aug_texts2=contents_aug2,
        labels=np.array(labels),
        fg_label=fg_label,
        aug_fg1=aug_fg1,
        aug_fg2=aug_fg2,
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=0)

def create_eval_loader(contents, labels, tokenizer, max_len, batch_size):
    """
    Creates a DataLoader for evaluation on original texts only.
    """
    ds = NewsDataset(texts=contents, labels=np.array(labels), tokenizer=tokenizer, max_len=max_len)
    return DataLoader(ds, batch_size=batch_size, num_workers=0)

## 🔢 8. Seed Setter for Reproducibility

Ensures consistent results across training iterations by fixing all random seeds.

In [ ]:
def set_seed(seed):
    """
    Set all random seeds for deterministic behavior.
    """
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    print(f"🔁 Seed set to {seed}")

## 🏋️ 9. Training Function: Main Training Loop

Trains the RoBERTa model using:
- Supervised classification loss
- Consistency loss (KL divergence between original and augmented outputs)
- Fine-grained framing loss (BCE)

Evaluates on 5 test sets: original + 4 adversarial styles.
Saves model checkpoints and logs metrics.

In [ ]:
def train_model(tokenizer, max_len, n_epochs, batch_size, datasetname, iter):
    """
    Main training loop for one iteration.
    Loads data, trains model, evaluates on multiple test sets, saves model.
    """
    print(f"\n📈 Starting Training Iteration {iter+1}/{args.iters} for dataset: {datasetname}")
    
    # Load dataset: original and adversarial splits
    x_train, x_test, x_test_a, x_test_b, x_test_c, x_test_d, y_train, y_test = load_articles(datasetname)
    
    # Load fine-grained framing labels
    x_train_res1, x_train_res2, y_train_fg, y_train_fg_m, y_train_fg_t = load_reframing(datasetname)
    
    # Create data loaders
    test_loader = create_eval_loader(x_test, y_test, tokenizer, max_len, batch_size)
    test_loader_a = create_eval_loader(x_test_a, y_test, tokenizer, max_len, batch_size)
    test_loader_b = create_eval_loader(x_test_b, y_test, tokenizer, max_len, batch_size)
    test_loader_c = create_eval_loader(x_test_c, y_test, tokenizer, max_len, batch_size)
    test_loader_d = create_eval_loader(x_test_d, y_test, tokenizer, max_len, batch_size)

    # Initialize model
    model = RobertaClassifier(n_classes=4).to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)
    total_steps = 10000  # Approximate total training steps
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    train_losses = []
    train_accs = []

    for epoch in range(n_epochs):
        model.train()
        
        # Create training loader with stochastic synonym augmentation
        train_loader = create_train_loader(
            x_train, x_train_res1, x_train_res2, y_train, y_train_fg, y_train_fg_m, y_train_fg_t, tokenizer, max_len, batch_size
        )
        
        avg_loss = []
        avg_acc = []
        
        for batch_data in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
            # Move data to device
            input_ids = batch_data["input_ids"].to(device)
            attention_mask = batch_data["attention_mask"].to(device)
            input_ids_aug1 = batch_data["input_ids_aug1"].to(device)
            attention_mask_aug1 = batch_data["attention_mask_aug1"].to(device)
            input_ids_aug2 = batch_data["input_ids_aug2"].to(device)
            attention_mask_aug2 = batch_data["attention_mask_aug2"].to(device)
            targets = batch_data["labels"].to(device)
            fg_labels = batch_data["fg_label"].to(device)
            fg_labels_aug1 = batch_data["fg_label_aug1"].to(device)
            fg_labels_aug2 = batch_data["fg_label_aug2"].to(device)
            
            # Forward pass
            out_labels, out_labels_bi = model(input_ids=input_ids, attention_mask=attention_mask)
            out_labels_aug1, out_labels_bi_aug1 = model(input_ids=input_ids_aug1, attention_mask=attention_mask_aug1)
            out_labels_aug2, out_labels_bi_aug2 = model(input_ids=input_ids_aug2, attention_mask=attention_mask_aug2)
            
            # Loss Components
            # 1. Fine-grained framing loss (BCE on sigmoid outputs)
            fg_criterion = nn.BCELoss()
            finegrain_loss = (
                fg_criterion(torch.sigmoid(out_labels_bi), fg_labels) +
                fg_criterion(torch.sigmoid(out_labels_bi_aug1), fg_labels_aug1) +
                fg_criterion(torch.sigmoid(out_labels_bi_aug2), fg_labels_aug2)
            ) / 3
            
            # 2. Supervised classification loss (CrossEntropy)
            sup_criterion = nn.CrossEntropyLoss()
            sup_loss = sup_criterion(out_labels_bi, targets)
            
            # 3. Consistency regularization (KL divergence between original and augmented predictions)
            out_probs = F.softmax(out_labels_bi, dim=-1)
            aug_log_prob1 = F.log_softmax(out_labels_bi_aug1, dim=-1)
            aug_log_prob2 = F.log_softmax(out_labels_bi_aug2, dim=-1)
            
            cons_criterion = nn.KLDivLoss(reduction='batchmean')
            cons_loss = 0.5 * cons_criterion(aug_log_prob1, out_probs) + 0.5 * cons_criterion(aug_log_prob2, out_probs)
            
            # Total loss
            loss = sup_loss + cons_loss + finegrain_loss
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()
            scheduler.step()
            
            # Accuracy calculation
            _, pred = out_labels_bi.max(dim=-1)
            correct = pred.eq(targets).sum().item()
            train_acc = correct / len(targets)
            
            avg_loss.append(loss.item())
            avg_acc.append(train_acc)
        
        # Log epoch metrics
        train_losses.append(np.mean(avg_loss))
        train_accs.append(np.mean(avg_acc))
        print(f"Iter {iter:03d} | Epoch {epoch+1:05d} | Train Loss {np.mean(avg_loss):.4f} | Train Acc. {np.mean(avg_acc):.4f}")
        
        # Evaluate on test sets at last epoch
        if epoch == n_epochs - 1:
            model.eval()
            y_pred = []
            y_pred_a = []
            y_pred_b = []
            y_pred_c = []
            y_pred_d = []
            y_test_true = []
            
            # Evaluate on original test set
            for batch_data in tqdm(test_loader, desc="Evaluating Original"):
                with torch.no_grad():
                    input_ids = batch_data["input_ids"].to(device)
                    attention_mask = batch_data["attention_mask"].to(device)
                    targets = batch_data["labels"].to(device)
                    _, val_out = model(input_ids=input_ids, attention_mask=attention_mask)
                    _, val_pred = val_out.max(dim=1)
                    y_pred.append(val_pred)
                    y_test_true.append(targets)
            
            # Evaluate on adversarial set A: Objective
            for batch_data in tqdm(test_loader_a, desc="Evaluating A (Objective)"):
                with torch.no_grad():
                    input_ids = batch_data["input_ids"].to(device)
                    attention_mask = batch_data["attention_mask"].to(device)
                    _, val_out = model(input_ids=input_ids, attention_mask=attention_mask)
                    _, val_pred = val_out.max(dim=1)
                    y_pred_a.append(val_pred)
            
            # Evaluate on adversarial set B: Neutral
            for batch_data in tqdm(test_loader_b, desc="Evaluating B (Neutral)"):
                with torch.no_grad():
                    input_ids = batch_data["input_ids"].to(device)
                    attention_mask = batch_data["attention_mask"].to(device)
                    _, val_out = model(input_ids=input_ids, attention_mask=attention_mask)
                    _, val_pred = val_out.max(dim=1)
                    y_pred_b.append(val_pred)
            
            # Evaluate on adversarial set C: Emotionally Triggering
            for batch_data in tqdm(test_loader_c, desc="Evaluating C (Emotionally Triggering)"):
                with torch.no_grad():
                    input_ids = batch_data["input_ids"].to(device)
                    attention_mask = batch_data["attention_mask"].to(device)
                    _, val_out = model(input_ids=input_ids, attention_mask=attention_mask)
                    _, val_pred = val_out.max(dim=1)
                    y_pred_c.append(val_pred)
            
            # Evaluate on adversarial set D: Sensational
            for batch_data in tqdm(test_loader_d, desc="Evaluating D (Sensational)"):
                with torch.no_grad():
                    input_ids = batch_data["input_ids"].to(device)
                    attention_mask = batch_data["attention_mask"].to(device)
                    _, val_out = model(input_ids=input_ids, attention_mask=attention_mask)
                    _, val_pred = val_out.max(dim=1)
                    y_pred_d.append(val_pred)
            
            # Concatenate predictions
            y_pred = torch.cat(y_pred, dim=0)
            y_pred_a = torch.cat(y_pred_a, dim=0)
            y_pred_b = torch.cat(y_pred_b, dim=0)
            y_pred_c = torch.cat(y_pred_c, dim=0)
            y_pred_d = torch.cat(y_pred_d, dim=0)
            y_test_true = torch.cat(y_test_true, dim=0)
            
            # Compute metrics
            acc = accuracy_score(y_test_true.cpu().numpy(), y_pred.cpu().numpy())
            precision, recall, fscore, _ = score(y_test_true.cpu().numpy(), y_pred.cpu().numpy(), average='macro')
            
            acc_a = accuracy_score(y_test_true.cpu().numpy(), y_pred_a.cpu().numpy())
            precision_a, recall_a, fscore_a, _ = score(y_test_true.cpu().numpy(), y_pred_a.cpu().numpy(), average='macro')
            
            acc_b = accuracy_score(y_test_true.cpu().numpy(), y_pred_b.cpu().numpy())
            precision_b, recall_b, fscore_b, _ = score(y_test_true.cpu().numpy(), y_pred_b.cpu().numpy(), average='macro')
            
            acc_c = accuracy_score(y_test_true.cpu().numpy(), y_pred_c.cpu().numpy())
            precision_c, recall_c, fscore_c, _ = score(y_test_true.cpu().numpy(), y_pred_c.cpu().numpy(), average='macro')
            
            acc_d = accuracy_score(y_test_true.cpu().numpy(), y_pred_d.cpu().numpy())
            precision_d, recall_d, fscore_d, _ = score(y_test_true.cpu().numpy(), y_pred_d.cpu().numpy(), average='macro')
            
            # Average across adversarial sets
            acc_res = (acc_a + acc_b + acc_c + acc_d) / 4
            precision_res = (precision_a + precision_b + precision_c + precision_d) / 4
            recall_res = (recall_a + recall_b + recall_c + recall_d) / 4
            fscore_res = (fscore_a + fscore_b + fscore_c + fscore_d) / 4
            
    # Save model checkpoint
    os.makedirs('checkpoints', exist_ok=True)
    checkpoint_path = os.path.join('checkpoints', f'{datasetname}_synonym_iter{iter}.m')
    torch.save(model.state_dict(), checkpoint_path)
    print(f"💾 Model saved to: {checkpoint_path}")

    # Print results
    print(f"\n----------------- End of Iter {iter:03d} -----------------")
    print("🎯 Original Test Set:")
    print([f'Global Test Accuracy: {acc:.4f}',
           f'Precision: {precision:.4f}',
           f'Recall: {recall:.4f}',
           f'F1: {fscore:.4f}'])
    
    print("🟦 Adversarial (A - Objective):")
    print([f'Global Test Accuracy: {acc_a:.4f}',
           f'Precision: {precision_a:.4f}',
           f'Recall: {recall_a:.4f}',
           f'F1: {fscore_a:.4f}'])
    
    print("🟩 Adversarial (B - Neutral):")
    print([f'Global Test Accuracy: {acc_b:.4f}',
           f'Precision: {precision_b:.4f}',
           f'Recall: {recall_b:.4f}',
           f'F1: {fscore_b:.4f}'])
    
    print("🟨 Adversarial (C - Emotionally Triggering):")
    print([f'Global Test Accuracy: {acc_c:.4f}',
           f'Precision: {precision_c:.4f}',
           f'Recall: {recall_c:.4f}',
           f'F1: {fscore_c:.4f}'])
    
    print("🟥 Adversarial (D - Sensational):")
    print([f'Global Test Accuracy: {acc_d:.4f}',
           f'Precision: {precision_d:.4f}',
           f'Recall: {recall_d:.4f}',
           f'F1: {fscore_d:.4f}'])
    
    print("⚪ Adversarial (Average across A–D):")
    print([f'Global Test Accuracy: {acc_res:.4f}',
           f'Precision: {precision_res:.4f}',
           f'Recall: {recall_res:.4f}',
           f'F1: {fscore_res:.4f}'])

    return (
        acc, precision, recall, fscore,
        acc_res, precision_res, recall_res, fscore_res,
        acc_a, precision_a, recall_a, fscore_a,
        acc_b, precision_b, recall_b, fscore_b,
        acc_c, precision_c, recall_c, fscore_c,
        acc_d, precision_d, recall_d, fscore_d
    )

## 📊 10. Main Training Execution Loop

Runs multiple training iterations (with different seeds) to report robust average performance.
Logs final metrics to a text file for analysis.

In [ ]:
def main():
    """
    Main execution function: runs multiple training iterations and aggregates results.
    """
    datasetname = args.dataset_name
    batch_size = args.batch_size
    max_len = 512
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    n_epochs = args.n_epochs
    iterations = args.iters

    # Store results for all iterations
    test_accs = []
    prec_all, rec_all, f1_all = [], [], []
    test_accs_a, prec_all_a, rec_all_a, f1_all_a = [], [], [], []
    test_accs_b, prec_all_b, rec_all_b, f1_all_b = [], [], [], []
    test_accs_c, prec_all_c, rec_all_c, f1_all_c = [], [], [], []
    test_accs_d, prec_all_d, rec_all_d, f1_all_d = [], [], [], []
    test_accs_res = []
    prec_all_res, rec_all_res, f1_all_res = [], [], []

    print(f"\n🚀 Starting {iterations} training iterations for dataset '{datasetname}'")

    for iter in range(iterations):
        set_seed(iter)
        metrics = train_model(tokenizer, max_len, n_epochs, batch_size, datasetname, iter)
        
        (acc, prec, recall, f1,
         acc_res, prec_res, recall_res, f1_res,
         acc_a, prec_a, recall_a, f1_a,
         acc_b, prec_b, recall_b, f1_b,
         acc_c, prec_c, recall_c, f1_c,
         acc_d, prec_d, recall_d, f1_d) = metrics

        # Store metrics
        test_accs.append(acc)
        prec_all.append(prec)
        rec_all.append(recall)
        f1_all.append(f1)
        
        test_accs_res.append(acc_res)
        prec_all_res.append(prec_res)
        rec_all_res.append(recall_res)
        f1_all_res.append(f1_res)
        
        test_accs_a.append(acc_a)
        prec_all_a.append(prec_a)
        rec_all_a.append(recall_a)
        f1_all_a.append(f1_a)
        
        test_accs_b.append(acc_b)
        prec_all_b.append(prec_b)
        rec_all_b.append(recall_b)
        f1_all_b.append(f1_b)
        
        test_accs_c.append(acc_c)
        prec_all_c.append(prec_c)
        rec_all_c.append(recall_c)
        f1_all_c.append(f1_c)
        
        test_accs_d.append(acc_d)
        prec_all_d.append(prec_d)
        rec_all_d.append(recall_d)
        f1_all_d.append(f1_d)

    # Print final aggregated results
    print("\n" + "="*60)
    print("📊 FINAL RESULTS ACROSS ALL ITERATIONS")
    print("="*60)
    
    print(f"✅ Original Test: Acc={sum(test_accs)/iterations:.4f} | Prec={sum(prec_all)/iterations:.4f} | Rec={sum(rec_all)/iterations:.4f} | F1={sum(f1_all)/iterations:.4f}")
    print(f"🔵 Adversarial A (Objective): Acc={sum(test_accs_a)/iterations:.4f} | Prec={sum(prec_all_a)/iterations:.4f} | Rec={sum(rec_all_a)/iterations:.4f} | F1={sum(f1_all_a)/iterations:.4f}")
    print(f"🟢 Adversarial B (Neutral): Acc={sum(test_accs_b)/iterations:.4f} | Prec={sum(prec_all_b)/iterations:.4f} | Rec={sum(rec_all_b)/iterations:.4f} | F1={sum(f1_all_b)/iterations:.4f}")
    print(f"🟡 Adversarial C (Emotional): Acc={sum(test_accs_c)/iterations:.4f} | Prec={sum(prec_all_c)/iterations:.4f} | Rec={sum(rec_all_c)/iterations:.4f} | F1={sum(f1_all_c)/iterations:.4f}")
    print(f"🔴 Adversarial D (Sensational): Acc={sum(test_accs_d)/iterations:.4f} | Prec={sum(prec_all_d)/iterations:.4f} | Rec={sum(rec_all_d)/iterations:.4f} | F1={sum(f1_all_d)/iterations:.4f}")
    print(f"⚪ Adversarial Average: Acc={sum(test_accs_res)/iterations:.4f} | Prec={sum(prec_all_res)/iterations:.4f} | Rec={sum(rec_all_res)/iterations:.4f} | F1={sum(f1_all_res)/iterations:.4f}")

    # Save detailed logs
    os.makedirs('logs', exist_ok=True)
    log_file = os.path.join('logs', f'log_{datasetname}_{args.model_name}_iter{iterations}.txt')
    
    with open(log_file, 'a+') as f:
        f.write('\n' + '='*80 + '\n')
        f.write(f"📊 EXPERIMENT: {datasetname} | Model: {args.model_name} | Iters: {iterations} | Epochs: {n_epochs} | Batch: {batch_size}\n")
        f.write('='*80 + '\n\n')
        
        f.write('------------- ORIGINAL TEST SET -------------\n')
        f.write(f'All Acc.s: {test_accs}\n')
        f.write(f'All Prec.s: {prec_all}\n')
        f.write(f'All Rec.s: {rec_all}\n')
        f.write(f'All F1.s: {f1_all}\n')
        f.write(f'Average Acc.: {sum(test_accs)/iterations:.4f}\n')
        f.write(f'Average Macro: Prec={sum(prec_all)/iterations:.4f}, Rec={sum(rec_all)/iterations:.4f}, F1={sum(f1_all)/iterations:.4f}\n\n')
        
        f.write('------------- ADVERSARIAL A (OBJECTIVE) -------------\n')
        f.write(f'All Acc.s: {test_accs_a}\n')
        f.write(f'All Prec.s: {prec_all_a}\n')
        f.write(f'All Rec.s: {rec_all_a}\n')
        f.write(f'All F1.s: {f1_all_a}\n')
        f.write(f'Average Acc.: {sum(test_accs_a)/iterations:.4f}\n')
        f.write(f'Average Macro: Prec={sum(prec_all_a)/iterations:.4f}, Rec={sum(rec_all_a)/iterations:.4f}, F1={sum(f1_all_a)/iterations:.4f}\n\n')
        
        f.write('------------- ADVERSARIAL B (NEUTRAL) -------------\n')
        f.write(f'All Acc.s: {test_accs_b}\n')
        f.write(f'All Prec.s: {prec_all_b}\n')
        f.write(f'All Rec.s: {rec_all_b}\n')
        f.write(f'All F1.s: {f1_all_b}\n')
        f.write(f'Average Acc.: {sum(test_accs_b)/iterations:.4f}\n')
        f.write(f'Average Macro: Prec={sum(prec_all_b)/iterations:.4f}, Rec={sum(rec_all_b)/iterations:.4f}, F1={sum(f1_all_b)/iterations:.4f}\n\n')
        
        f.write('------------- ADVERSARIAL C (EMOTIONALLY TRIGGERING) -------------\n')
        f.write(f'All Acc.s: {test_accs_c}\n')
        f.write(f'All Prec.s: {prec_all_c}\n')
        f.write(f'All Rec.s: {rec_all_c}\n')
        f.write(f'All F1.s: {f1_all_c}\n')
        f.write(f'Average Acc.: {sum(test_accs_c)/iterations:.4f}\n')
        f.write(f'Average Macro: Prec={sum(prec_all_c)/iterations:.4f}, Rec={sum(rec_all_c)/iterations:.4f}, F1={sum(f1_all_c)/iterations:.4f}\n\n')
        
        f.write('------------- ADVERSARIAL D (SENSATIONAL) -------------\n')
        f.write(f'All Acc.s: {test_accs_d}\n')
        f.write(f'All Prec.s: {prec_all_d}\n')
        f.write(f'All Rec.s: {rec_all_d}\n')
        f.write(f'All F1.s: {f1_all_d}\n')
        f.write(f'Average Acc.: {sum(test_accs_d)/iterations:.4f}\n')
        f.write(f'Average Macro: Prec={sum(prec_all_d)/iterations:.4f}, Rec={sum(rec_all_d)/iterations:.4f}, F1={sum(f1_all_d)/iterations:.4f}\n\n')
        
        f.write('------------- ADVERSARIAL AVERAGE (A–D) -------------\n')
        f.write(f'All Acc.s: {test_accs_res}\n')
        f.write(f'All Prec.s: {prec_all_res}\n')
        f.write(f'All Rec.s: {rec_all_res}\n')
        f.write(f'All F1.s: {f1_all_res}\n')
        f.write(f'Average Acc.: {sum(test_accs_res)/iterations:.4f}\n')
        f.write(f'Average Macro: Prec={sum(prec_all_res)/iterations:.4f}, Rec={sum(rec_all_res)/iterations:.4f}, F1={sum(f1_all_res)/iterations:.4f}\n')
    
    print(f"\n📝 Log saved to: {log_file}")

## ▶️ 11. Execute Training

This is the entry point of the script. Runs the main training loop.
Only executed if the script is run directly (not imported).

In [ ]:
if __name__ == "__main__":
    main()